In [55]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
pd.options.display.float_format = '{:.2f}'.format

In [56]:
#reading data
dfv = pd.read_csv('dfV_original_2.csv')
dfc = pd.read_csv('dfC_orignial.csv')

In [57]:
#create a new column based on the value of Platform variable
dfv['MediaType'] = dfv['Publisher'].apply(lambda x: 'Hybrid' if x in ['The Washington Post', 'NPR'] else 'Legacy' if x in ['CNN', 'Tampa Bay Times'] else 'SMonly')
dfc['MediaType'] = dfc['Publisher'].apply(lambda x: 'Hybrid' if x in ['The Washington Post', 'NPR'] else 'Legacy' if x in ['CNN', 'Tampa Bay Times'] else 'SMonly')

In [58]:
#create a dataset without duplicates
dfv_Dupeless = dfv.dropna(subset=['Graphics'])

In [59]:
#for dataset WITH duplicate, filling na values with the previous value
# BEGIN: Fill 'N/A' values with the value from another row that has the same 'VideoURL' but does not have 'N/A'
NAlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Fairness', 'Agenda', 'Media Intergrity', 'Decorum', 'Language', 'Topic', 'Presentation', 'ExternalVidSource']
for i in NAlist:
    dfv[i].fillna(9999, inplace=True)
for column in dfv.columns:
    dfv[column] = dfv.groupby('VideoURL')[column].transform(lambda x: x.replace(9999, x[x != 9999].iloc[0] if any(x != 9999) else 9999))

In [60]:
#reverse coded variables
dfv2 = dfv.copy()
dfv2['Fairness'] = 6-dfv.Fairness
dfv2['Agenda'] = 6-dfv['Agenda']
dfv2['Media Intergrity'] = 6-dfv['Media Intergrity']
dfv2['Decorum'] = 6-dfv['Decorum']
dfv2['Language'] = 6-dfv['Language']
#add Animation variable based on values in dfv. if Graphics is 1, then Animation is 1, otherwise 
dfv2['Animation'] = dfv['Graphics'].apply(lambda x: 1 if x == 1 else 0)
dfv2['Filters'] = dfv['Graphics'].apply(lambda x: 1 if x == 1 else 0)
#creating a 0 to 1 variable for some variables
dfv2['Decorum_Z']=(dfv['Decorum']-1)/4
dfv2['Language_Z']=(dfv['Language']-1)/4

dfv_Dupeless2 = dfv_Dupeless.copy()
dfv_Dupeless2['Fairness'] = 6 - dfv_Dupeless['Fairness']
dfv_Dupeless2['Agenda'] = 6 - dfv_Dupeless['Agenda']
dfv_Dupeless2['Media Intergrity'] = 6 - dfv_Dupeless['Media Intergrity']
dfv_Dupeless2['Decorum'] = 6 - dfv_Dupeless['Decorum'] 
dfv_Dupeless2['Language'] = 6 - dfv_Dupeless['Language']
#add Animation variable based on values in dfv. if Graphics is 1, then Animation is 1, otherwise 
dfv_Dupeless2['Animation'] = dfv_Dupeless['Graphics'].apply(lambda x: 1 if x == 1 else 0)
dfv_Dupeless2['Filters'] = dfv_Dupeless['Graphics'].apply(lambda x: 1 if x == 1 else 0)
#creating a 0 to 1 variable for some variables
dfv_Dupeless2['Decorum_Z']=(dfv_Dupeless['Decorum']-1)/4
dfv_Dupeless2['Language_Z']=(dfv_Dupeless['Language']-1)/4

#overiting the original datasets
dfv=dfv2
dfv_Dupeless=dfv_Dupeless2

dfv_Dupeless.rename(columns={'Media Intergrity': 'MediaIntegrity'}, inplace=True)
dfv_Dupeless.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfv.rename(columns={'Media Intergrity': 'MediaIntegrity'}, inplace=True)
dfv.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfc.rename(columns={'Likes/Hearts': 'CommLikes'}, inplace=True)
dfc.rename(columns={'Comment': 'CommentText'}, inplace=True)

In [61]:
#recode variables with value labels
dfv_recode = dfv.copy()
dfv_recode['Shots'] = dfv['Shots'].replace({1: 'steady', 0: 'not steady', 99: np.nan})
dfv_recode['Graphics'] = dfv['Graphics'].replace({1: 'animations', 2: 'filters', 3: 'both', 4: 'none', 99: np.nan})
dfv_recode['EditStyle'] = dfv['EditsStyle'].replace({1: 'clean', 0: 'not clean', 99: np.nan})
dfv_recode['Broll'] = dfv['Broll'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_recode['NatSound'] = dfv['NatSound'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_recode['Audio'] = dfv['Audio'].replace({1: 'good', 0: 'poor', 99: np.nan})
dfv_recode['Music'] = dfv['Music'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_recode['Narration'] = dfv['Narration'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_recode['Humor'] = dfv['Humor'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_recode['Offensive'] = dfv['Offensive'].replace({1: 'not offensive', 2: 'atrocities', 3: 'pornography', 4: 'hateful', 5: 'other', 99: np.nan})
dfv_recode['Sourcing'] = dfv['Sourcing'].replace({1: 'use', 0: 'not use', 99: np.nan})
dfv_recode['Topic'] = dfv['Topic'].replace({1: 'West Bank Conflict', 2: 'Political', 3: 'Disaster/Accident', 4: 'Entertainment/Pop Culture', 5: 'Technology/AI', 6: 'Economics', 7: 'Environmental/Health', 8: 'Other'})

dfv_Dupeless_recode = dfv_Dupeless.copy()
dfv_Dupeless_recode['Shots'] = dfv_Dupeless['Shots'].replace({1: 'steady', 0: 'not steady', 99: np.nan})
dfv_Dupeless_recode['Graphics'] = dfv_Dupeless['Graphics'].replace({1: 'animations', 2: 'filters', 3: 'both', 4: 'none', 99: np.nan})
dfv_Dupeless_recode['EditStyle'] = dfv_Dupeless['EditsStyle'].replace({1: 'clean', 0: 'not clean', 99: np.nan})
dfv_Dupeless_recode['Broll'] = dfv_Dupeless['Broll'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_Dupeless_recode['NatSound'] = dfv_Dupeless['NatSound'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_Dupeless_recode['Audio'] = dfv_Dupeless['Audio'].replace({1: 'good', 0: 'poor', 99: np.nan})
dfv_Dupeless_recode['Music'] = dfv_Dupeless['Music'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_Dupeless_recode['Narration'] = dfv_Dupeless['Narration'].replace({1: 'includes', 0: 'lacks', 99: np.nan})
dfv_Dupeless_recode['Humor'] = dfv_Dupeless['Humor'].replace({1: 'yes', 0: 'no', 99: np.nan})
dfv_Dupeless_recode['Offensive'] = dfv_Dupeless['Offensive'].replace({1: 'not offensive', 2: 'atrocities', 3: 'pornography', 4: 'hateful', 5: 'other', 99: np.nan})
dfv_Dupeless_recode['Sourcing'] = dfv_Dupeless['Sourcing'].replace({1: 'use', 0: 'not use', 99: np.nan})
dfv_Dupeless_recode['Topic'] = dfv_Dupeless['Topic'].replace({1: 'West Bank Conflict', 2: 'Political', 3: 'Disaster/Accident', 4: 'Entertainment/Pop Culture', 5: 'Technology/AI', 6: 'Economics', 7: 'Environmental/Health', 8: 'Other'})

In [62]:
dfv_ColNames = dfv_recode
NAlist2 = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in NAlist2:
    dfv_ColNames.loc[:,i] = dfv_ColNames[i].astype(str) + ' ({})'.format(i)

dfv_Dupeless_ColNames = dfv_Dupeless_recode
NAlist2 = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in NAlist2:
    dfv_Dupeless_ColNames.loc[:,i] = dfv_Dupeless_ColNames[i].astype(str) + ' ({})'.format(i)

In [63]:
#recade variables with value labels
# dfc['Focus'] = dfc['Focus'].replace({1: 'central',  0: 'peripheral', 99: 'NA'})
# change 99 to NA in dfc
dfc = dfc.replace({99: np.nan})
# change -93 to NA in dfv
dfv = dfv.replace({-93: np.nan})
dfv = dfv.replace({99: np.nan})

In [64]:
#saving files
dfv.to_csv('Analysis/dfv.csv', index=False)
dfv_Dupeless.to_csv('Analysis/dfv_noDupe.csv', index=False)
dfv_recode.to_csv('Analysis/dfv_recode.csv', index=False)
dfv_Dupeless_recode.to_csv('Analysis/dfv_noDupe_recode.csv', index=False)
dfv_ColNames.to_csv('Analysis/dfv_ColNames.csv', index=False)
dfv_Dupeless_ColNames.to_csv('Analysis/dfv_noDupe_ColNames.csv', index=False)
dfc.to_csv('Analysis/dfc.csv', index=False)

In [65]:

IVlist = ['Platform', 'Publisher', 'MediaType']
DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']

For each DV, define which value is Trendy (1) and which value is Traditional (0).
Add up the variables to create a Composite variable, Trendiness.
You dont have to have all the DVs for the composite variable.
But they have to be 3 or more.

Cronbach's Alpha.
Test for reliability of composite variables
>.80, reliable

In [66]:
# # Testing different between Publisher
# # Specify the subfolder path
# subfolder_path = 'Publisher/'
# # Create the subfolder if it doesn't exist
# if not os.path.exists(subfolder_path):
#     os.makedirs(subfolder_path)
# # Variables to test
# DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
# for i in DVlist:
#     tab1=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'], margins=True)
#     tab1.reset_index(inplace=True)
#     tab1.columns.name = None
#     tab1p=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'], normalize='columns').round(4)*100
#     tab1p.reset_index(inplace=True)
#     tab1p.columns.name = None
#     tab1a=pd.merge(tab1, tab1p, on=i, how='left')
#     globals()[f"CrossTab_{i}"] = tab1a
#     test=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'])# crosstab with raw numbers
#     chi2, p, dof, expected = stats.chi2_contingency(test)
#     print(i+": " + "chi2 = "+ chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
#     # Save the output file in the subfolder
#     output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
#     tab1a.to_csv(output_file_path, index=False)

In [67]:
# # Testing different between Platform
# # Specify the subfolder path
# subfolder_path = 'Platform/'
# # Create the subfolder if it doesn't exist
# if not os.path.exists(subfolder_path):
#     os.makedirs(subfolder_path)
# # Variables to test
# DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
# for i in DVlist:
#     tab1=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'], margins=True)
#     tab1.reset_index(inplace=True)
#     tab1.columns.name = None
#     tab1p=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'], normalize='columns').round(4)*100
#     tab1p.reset_index(inplace=True)
#     tab1p.columns.name = None
#     tab1a=pd.merge(tab1, tab1p, on=i, how='left')
#     globals()[f"CrossTab_{i}"] = tab1a
#     test=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'])# crosstab with raw numbers
#     chi2, p, dof, expected = stats.chi2_contingency(test)
#     print(i+": " + "chi2 = "+ chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
#     # Save the output file in the subfolder
#     output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
#     tab1a.to_csv(output_file_path, index=False)

In [68]:
# # Testing different between MediaType
# # Specify the subfolder path
# subfolder_path = 'MediaType/'
# # Create the subfolder if it doesn't exist
# if not os.path.exists(subfolder_path):
#     os.makedirs(subfolder_path)
# # Variables to test
# DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
# for i in DVlist:
#     tab1=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'], margins=True)
#     tab1.reset_index(inplace=True)
#     tab1.columns.name = None
#     tab1p=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'], normalize='columns').round(4)*100
#     tab1p.reset_index(inplace=True)
#     tab1p.columns.name = None
#     tab1a=pd.merge(tab1, tab1p, on=i, how='left')
#     globals()[f"CrossTab_{i}"] = tab1a
#     test=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'])# crosstab with raw numbers
#     chi2, p, dof, expected = stats.chi2_contingency(test)
#     print(i + ": " + "chi2 = " + chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
#     # Save the output file in the subfolder
#     output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
#     tab1a.to_csv(output_file_path, index=False)

In [69]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_Dupeless_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,MediaType,2,123,1.08,0.34,0.02
1,Comments,MediaType,2,123,1.84,0.16,0.03
2,MediaIntegrity,MediaType,2,123,12.34,0.00,0.17
3,Presentation,MediaType,2,123,75.33,0.00,0.55


In [70]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,MediaType,Hybrid,Legacy,False,True,0.49,82.00,two-sided,0.63,1.00,bonf,0.253,0.11
1,Likes,MediaType,Hybrid,SMonly,False,True,-0.82,82.00,two-sided,0.42,1.00,bonf,0.305,-0.18
2,Likes,MediaType,Legacy,SMonly,False,True,-1.95,82.00,two-sided,0.05,0.16,bonf,1.179,-0.42
3,Comments,MediaType,Hybrid,Legacy,False,True,-1.74,82.00,two-sided,0.09,0.26,bonf,0.841,-0.38
4,Comments,MediaType,Hybrid,SMonly,False,True,-1.31,82.00,two-sided,0.19,0.58,bonf,0.479,-0.28
5,Comments,MediaType,Legacy,SMonly,False,True,0.90,82.00,two-sided,0.37,1.00,bonf,0.325,0.20
6,MediaIntegrity,MediaType,Hybrid,Legacy,False,True,-1.24,82.00,two-sided,0.22,0.66,bonf,0.444,-0.27
7,MediaIntegrity,MediaType,Hybrid,SMonly,False,True,3.49,82.00,two-sided,0.00,0.00,bonf,37.661,0.76
8,MediaIntegrity,MediaType,Legacy,SMonly,False,True,4.72,82.00,two-sided,0.00,0.00,bonf,1780.57,1.02
9,Presentation,MediaType,Hybrid,Legacy,False,True,4.97,82.00,two-sided,0.00,0.00,bonf,4320.495,1.07


In [71]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Hybrid,17562.60,76140.58
1,Likes,Legacy,11207.12,36336.52
2,Likes,SMonly,28757.62,45576.91
3,Comments,Hybrid,348.07,623.04
4,Comments,Legacy,836.45,1711.96
5,Comments,SMonly,567.12,888.48
6,MediaIntegrity,Hybrid,3.48,0.80
7,MediaIntegrity,Legacy,3.69,0.78
8,MediaIntegrity,SMonly,2.83,0.88
9,Presentation,Hybrid,2.93,0.81


In [72]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_Dupeless_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,Publisher,5,120,1.02,0.41,0.04
1,Comments,Publisher,5,120,5.77,0.00,0.19
2,MediaIntegrity,Publisher,5,120,7.11,0.00,0.23
3,Presentation,Publisher,5,120,35.66,0.00,0.60


In [73]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,Publisher,CNN,GoodMorning BadNews,False,True,-0.66,40.00,two-sided,0.51,1.00,bonf,0.36,-0.20
1,Likes,Publisher,CNN,NPR,False,True,-0.18,40.00,two-sided,0.86,1.00,bonf,0.307,-0.05
2,Likes,Publisher,CNN,Tampa Bay Times,False,True,2.03,40.00,two-sided,0.05,0.74,bonf,1.505,0.61
3,Likes,Publisher,CNN,The Washington Post,False,True,1.25,40.00,two-sided,0.22,1.00,bonf,0.562,0.38
4,Likes,Publisher,CNN,UnderTheDeskNews,False,True,-0.22,40.00,two-sided,0.82,1.00,bonf,0.309,-0.07
5,Likes,Publisher,GoodMorning BadNews,NPR,False,True,0.21,40.00,two-sided,0.83,1.00,bonf,0.308,0.06
6,Likes,Publisher,GoodMorning BadNews,Tampa Bay Times,False,True,3.00,40.00,two-sided,0.00,0.07,bonf,9.005,0.91
7,Likes,Publisher,GoodMorning BadNews,The Washington Post,False,True,2.18,40.00,two-sided,0.03,0.52,bonf,1.936,0.66
8,Likes,Publisher,GoodMorning BadNews,UnderTheDeskNews,False,True,0.48,40.00,two-sided,0.64,1.00,bonf,0.332,0.14
9,Likes,Publisher,NPR,Tampa Bay Times,False,True,1.13,40.00,two-sided,0.27,1.00,bonf,0.502,0.34


In [74]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

In [75]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,Platform,2,123,3.50,0.03,0.05
1,Comments,Platform,2,123,4.06,0.02,0.06
2,MediaIntegrity,Platform,2,123,0.03,0.97,0.00
3,Presentation,Platform,2,123,0.14,0.87,0.00


In [76]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,Platform,Instagram Reels,TikTok,False,True,-0.27,82.00,two-sided,0.79,1.00,bonf,0.235,-0.06
1,Likes,Platform,Instagram Reels,YouTube Shorts,False,True,1.96,82.00,two-sided,0.05,0.16,bonf,1.199,0.42
2,Likes,Platform,TikTok,YouTube Shorts,False,True,4.14,82.00,two-sided,0.00,0.00,bonf,263.434,0.90
3,Comments,Platform,Instagram Reels,TikTok,False,True,0.69,82.00,two-sided,0.49,1.00,bonf,0.28,0.15
4,Comments,Platform,Instagram Reels,YouTube Shorts,False,True,2.55,82.00,two-sided,0.01,0.04,bonf,3.649,0.55
5,Comments,Platform,TikTok,YouTube Shorts,False,True,3.06,82.00,two-sided,0.00,0.01,bonf,12.09,0.66
6,MediaIntegrity,Platform,Instagram Reels,TikTok,False,True,-0.12,82.00,two-sided,0.90,1.00,bonf,0.229,-0.03
7,MediaIntegrity,Platform,Instagram Reels,YouTube Shorts,False,True,0.12,82.00,two-sided,0.91,1.00,bonf,0.229,0.03
8,MediaIntegrity,Platform,TikTok,YouTube Shorts,False,True,0.24,82.00,two-sided,0.81,1.00,bonf,0.233,0.05
9,Presentation,Platform,Instagram Reels,TikTok,False,True,-0.46,82.00,two-sided,0.65,1.00,bonf,0.249,-0.10


In [77]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Instagram Reels,26196.71,82502.02
1,Likes,TikTok,30128.95,45105.94
2,Likes,YouTube Shorts,1201.67,3358.39
3,Comments,Instagram Reels,884.40,1700.43
4,Comments,TikTok,679.00,911.47
5,Comments,YouTube Shorts,188.24,496.62
6,MediaIntegrity,Instagram Reels,3.33,0.90
7,MediaIntegrity,TikTok,3.36,0.85
8,MediaIntegrity,YouTube Shorts,3.31,0.95
9,Presentation,Instagram Reels,3.05,1.25


In [78]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
merged_anova = pd.DataFrame()
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova


,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,CommLikes,Platform,2,992,10.65,0.00,0.02
1,CharacterNum,Platform,2,988,45.61,0.00,0.08
2,CommLength,Platform,2,993,11.85,0.00,0.02
3,Sentiment,Platform,2,976,5.36,0.00,0.01
4,Focus,Platform,2,991,7.60,0.00,0.02
5,Alignment,Platform,2,537,2.88,0.06,0.01


In [79]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,CommLikes,MediaType,2,992,3.30,0.04,0.01
1,CharacterNum,MediaType,2,988,1.38,0.25,0.00
2,CommLength,MediaType,2,993,1.33,0.26,0.00
3,Sentiment,MediaType,2,976,16.38,0.00,0.03
4,Focus,MediaType,2,991,1.02,0.36,0.00
5,Alignment,MediaType,2,537,16.72,0.00,0.06


In [80]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,CommLikes,MediaType,Hybrid,Legacy,False,True,-1.94,465.24,two-sided,0.05,0.16,bonf,0.65,-0.15
1,CommLikes,MediaType,Hybrid,SMonly,False,True,-3.40,639.80,two-sided,0.00,0.00,bonf,26.011,-0.19
2,CommLikes,MediaType,Legacy,SMonly,False,True,-1.78,804.71,two-sided,0.08,0.23,bonf,0.379,-0.11
3,CharacterNum,MediaType,Hybrid,Legacy,False,True,1.65,308.92,two-sided,0.10,0.30,bonf,0.393,0.16
4,CharacterNum,MediaType,Hybrid,SMonly,False,True,0.51,338.26,two-sided,0.61,1.00,bonf,0.111,0.04
5,CharacterNum,MediaType,Legacy,SMonly,False,True,-1.45,788.10,two-sided,0.15,0.45,bonf,0.224,-0.10
6,CommLength,MediaType,Hybrid,Legacy,False,True,-0.06,329.40,two-sided,0.95,1.00,bonf,0.105,-0.01
7,CommLength,MediaType,Hybrid,SMonly,False,True,-1.17,282.62,two-sided,0.24,0.73,bonf,0.19,-0.11
8,CommLength,MediaType,Legacy,SMonly,False,True,-1.44,671.35,two-sided,0.15,0.45,bonf,0.221,-0.10
9,Sentiment,MediaType,Hybrid,Legacy,False,True,-4.91,366.80,two-sided,0.00,0.00,bonf,1.002e+04,-0.46


In [81]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,CommLikes,Hybrid,94.49,236.07
1,CommLikes,Legacy,162.07,525.87
2,CommLikes,SMonly,257.18,1002.81
3,CharacterNum,Hybrid,96.84,97.09
4,CharacterNum,Legacy,82.30,83.45
5,CharacterNum,SMonly,92.31,113.75
6,CommLength,Hybrid,2.21,0.82
7,CommLength,Legacy,2.22,0.75
8,CommLength,SMonly,2.30,0.77
9,Sentiment,Hybrid,2.34,1.05


In [82]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['CommLikes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,CommLikes,Publisher,5,989,11.49,0.00,0.05
1,CharacterNum,Publisher,5,985,1.12,0.35,0.01
2,CommLength,Publisher,5,990,0.70,0.63,0.00
3,Sentiment,Publisher,5,973,13.37,0.00,0.06
4,Focus,Publisher,5,988,3.91,0.00,0.02
5,Alignment,Publisher,5,534,8.56,0.00,0.07
